## Imports

In [12]:
import re
import os
import csv
import pandas as pd

In [13]:
from scripts import cleaner
from scripts import entro

In [14]:
from bs4 import BeautifulSoup
import urllib.request

## Data loading

In [15]:
decades = ['1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s']
decades_dict = {}

#### ignore

~~#### Read src csv lists to df's~~

In [ ]:
# # Takes in one of the decade CSV show lists and returns it as a formatted dataframe
# def csvToDataframe(filename):
#     df = pd.read_csv(filename, header=None, names=['title', 'start', 'end', 'wiki_url', 'transcript_url'])
#     return df

In [ ]:
# for decade in decades:
#     path = "src/" + decade[:-1] + "_shows.csv"
#     decades_dict[decade] = csvToDataframe(path)
#     # print(path + " has been processed")

In [ ]:
# shows = []
# for decade in decades:
#     curr_df = decades_dict[decade]
#     for i, row in curr_df.iterrows():
#         shows.append(row['title'])

#### Read metadata csv's to df's

In [20]:
meta = {}

In [61]:
for decade in decades:
    path = "metadata/episodes_" + decade[:-1] + ".csv"
    meta[decade] = pd.read_csv(path, header=0, converters={'season': str, 'episode': str})

#### Group df's by title and season

In [62]:
for decade in decades:
    curr_df = meta[decade]
    meta[decade] = curr_df.groupby(['title', 'season'])

In [63]:
meta['1950s']['episode'].count()

title        season
I Love Lucy  01        36
             02        32
             03        31
             04        30
             05        26
             06        27
Name: episode, dtype: int64

In [64]:
meta['2000s']['episode'].count()

title                  season
30 Rock                01        21
                       02        15
                       03        22
                       04        21
                       05        22
                       06        21
                       07        12
Big Bang Theory        01        17
                       02        23
                       03        23
                       04        24
                       05        24
                       06        24
                       07        24
                       08        24
                       09        24
                       10        24
                       11        24
                       12        24
Community              01        25
                       02        24
                       03        22
                       04        13
                       05        13
                       06        13
Malcolm in the Middle  01        16
                       02        2

## Retrieving transcripts

#### Functions for retrieval

In [65]:
a_mark = "{START OF EPISODE "
z_mark = "{END OF EPISODE "
base = "https://www.springfieldspringfield.co.uk/"

In [66]:
# Helper function for getTV()
def getScript(url):
    # Returns list of lines from episode transcript
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    text = soup.find("div", {"class":"scrolling-script-container"}).get_text()
    cleaned = cleaner.clean(text)
    return cleaned
    # return cleaner.split_into_sentences(cleaned)

In [67]:
# Pass in decade as a string value (the key for the current decade we're scraping)
def getTV(decade):
    for grp_idx, grp in meta[decade]:
        t_raw, season, ended = grp_idx[0], grp_idx[1], False
        if season < 7:
            t_clean = re.sub(r' ', '_', t_raw.lower())
            f_path = "transcripts/" + decade + "/" + t_clean + "_season" + str(season) + ".txt"
            # open file to begin writing
            f = open(f_path, "w")
            print("Currently scraping... " + t_raw + ", Season " + str(season))
            # loop through each episode of the current season
            for row_idx, row in grp.iterrows():
                # add episode start marker
                f.write(a_mark + str(row['episode']) + "}" + "\n")
                # get transcript
                text = getScript(base + row['url'])
                lines = cleaner.split_into_sentences(text)
                # add each line to the file
                for i in range(len(lines)):
                    f.write(lines[i])
                    if i < len(lines) - 1:
                        f.write("\n")
                # add episode end marker
                f.write("\n" + z_mark + str(row['episode']) + "}")
            # close the file !!!
            f.close()
            print("Finished scraping... " + t_raw + ", Season " + str(season))
        else:
            if not ended:
                print("Yeah I'm not scraping past 6 seasons bud... sorry.") #SixSeasonsAndAMovie
                ended = True

#### Retrieval by decade

In [37]:
# getTV('1950s')

Currently scraping... I Love Lucy, Season 1
Finished scraping... I Love Lucy, Season 1
Currently scraping... I Love Lucy, Season 2
Finished scraping... I Love Lucy, Season 2
Currently scraping... I Love Lucy, Season 3
Finished scraping... I Love Lucy, Season 3
Currently scraping... I Love Lucy, Season 4
Finished scraping... I Love Lucy, Season 4
Currently scraping... I Love Lucy, Season 5
Finished scraping... I Love Lucy, Season 5
Currently scraping... I Love Lucy, Season 6
Finished scraping... I Love Lucy, Season 6


In [39]:
# getTV('1970s')

Currently scraping... M*A*S*H, Season 1
Finished scraping... M*A*S*H, Season 1
Currently scraping... M*A*S*H, Season 2
Finished scraping... M*A*S*H, Season 2
Currently scraping... M*A*S*H, Season 3
Finished scraping... M*A*S*H, Season 3
Currently scraping... M*A*S*H, Season 4
Finished scraping... M*A*S*H, Season 4
Currently scraping... M*A*S*H, Season 5
Finished scraping... M*A*S*H, Season 5
Currently scraping... M*A*S*H, Season 6
Finished scraping... M*A*S*H, Season 6
Yeah I'm not scraping past 6 seasons bud... sorry.
Yeah I'm not scraping past 6 seasons bud... sorry.
Yeah I'm not scraping past 6 seasons bud... sorry.
Yeah I'm not scraping past 6 seasons bud... sorry.
Yeah I'm not scraping past 6 seasons bud... sorry.
Currently scraping... Mork and Mindy, Season 1
Finished scraping... Mork and Mindy, Season 1
Currently scraping... Mork and Mindy, Season 2
Finished scraping... Mork and Mindy, Season 2
Currently scraping... Mork and Mindy, Season 3
Finished scraping... Mork and Mindy, S

In [40]:
# getTV('1980s')

Currently scraping... Cheers, Season 1
Finished scraping... Cheers, Season 1
Currently scraping... Cheers, Season 2
Finished scraping... Cheers, Season 2
Currently scraping... Cheers, Season 3
Finished scraping... Cheers, Season 3
Currently scraping... Cheers, Season 4
Finished scraping... Cheers, Season 4
Currently scraping... Cheers, Season 5
Finished scraping... Cheers, Season 5
Currently scraping... Cheers, Season 6
Finished scraping... Cheers, Season 6
Yeah I'm not scraping past 6 seasons bud... sorry.
Yeah I'm not scraping past 6 seasons bud... sorry.
Yeah I'm not scraping past 6 seasons bud... sorry.
Yeah I'm not scraping past 6 seasons bud... sorry.
Yeah I'm not scraping past 6 seasons bud... sorry.
Currently scraping... Full House, Season 1
Finished scraping... Full House, Season 1
Currently scraping... Full House, Season 2
Finished scraping... Full House, Season 2
Currently scraping... Full House, Season 3
Finished scraping... Full House, Season 3
Currently scraping... Full H

In [41]:
# getTV('1990s')

Currently scraping... Boy Meets World, Season 1
Finished scraping... Boy Meets World, Season 1
Currently scraping... Boy Meets World, Season 2
Finished scraping... Boy Meets World, Season 2
Currently scraping... Boy Meets World, Season 3
Finished scraping... Boy Meets World, Season 3
Currently scraping... Boy Meets World, Season 4
Finished scraping... Boy Meets World, Season 4
Currently scraping... Boy Meets World, Season 5
Finished scraping... Boy Meets World, Season 5
Currently scraping... Boy Meets World, Season 6
Finished scraping... Boy Meets World, Season 6
Yeah I'm not scraping past 6 seasons bud... sorry.
Currently scraping... Buffy the Vampire Slayer, Season 1
Finished scraping... Buffy the Vampire Slayer, Season 1
Currently scraping... Buffy the Vampire Slayer, Season 2
Finished scraping... Buffy the Vampire Slayer, Season 2
Currently scraping... Buffy the Vampire Slayer, Season 3
Finished scraping... Buffy the Vampire Slayer, Season 3
Currently scraping... Buffy the Vampire 

In [42]:
# getTV('2000s')

Currently scraping... 30 Rock, Season 1
Finished scraping... 30 Rock, Season 1
Currently scraping... 30 Rock, Season 2
Finished scraping... 30 Rock, Season 2
Currently scraping... 30 Rock, Season 3
Finished scraping... 30 Rock, Season 3
Currently scraping... 30 Rock, Season 4
Finished scraping... 30 Rock, Season 4
Currently scraping... 30 Rock, Season 5
Finished scraping... 30 Rock, Season 5
Currently scraping... 30 Rock, Season 6
Finished scraping... 30 Rock, Season 6
Yeah I'm not scraping past 6 seasons bud... sorry.
Currently scraping... Big Bang Theory, Season 1
Finished scraping... Big Bang Theory, Season 1
Currently scraping... Big Bang Theory, Season 2
Finished scraping... Big Bang Theory, Season 2
Currently scraping... Big Bang Theory, Season 3
Finished scraping... Big Bang Theory, Season 3
Currently scraping... Big Bang Theory, Season 4
Finished scraping... Big Bang Theory, Season 4
Currently scraping... Big Bang Theory, Season 5
Finished scraping... Big Bang Theory, Season 5


In [43]:
# getTV('2010s')

Currently scraping... Brooklyn Nine-Nine, Season 1
Finished scraping... Brooklyn Nine-Nine, Season 1
Currently scraping... Brooklyn Nine-Nine, Season 2
Finished scraping... Brooklyn Nine-Nine, Season 2
Currently scraping... Brooklyn Nine-Nine, Season 3
Finished scraping... Brooklyn Nine-Nine, Season 3
Currently scraping... Brooklyn Nine-Nine, Season 4
Finished scraping... Brooklyn Nine-Nine, Season 4
Currently scraping... Brooklyn Nine-Nine, Season 5
Finished scraping... Brooklyn Nine-Nine, Season 5
Currently scraping... Brooklyn Nine-Nine, Season 6
Finished scraping... Brooklyn Nine-Nine, Season 6
Yeah I'm not scraping past 6 seasons bud... sorry.
Currently scraping... Fleabag, Season 1
Finished scraping... Fleabag, Season 1
Currently scraping... Fleabag, Season 2
Finished scraping... Fleabag, Season 2
Currently scraping... New Girl, Season 1
Finished scraping... New Girl, Season 1
Currently scraping... New Girl, Season 2
Finished scraping... New Girl, Season 2
Currently scraping... N

## Processing transcripts

#### Function for processing

In [68]:
pattern_all = re.compile(r'\{START OF EPISODE \d{1,2}\}(.+?)\{END OF EPISODE \d{1,2}\}', re.DOTALL)

In [69]:
pattern_50s = re.compile(r'\{START OF EPISODE \d{2}\}(.+?)\{END OF EPISODE \d{2}\}', re.DOTALL)

In [75]:
def obtainEntropy(decade):
    entropy = []
    for grp_idx, grp in meta[decade]:
        t_raw, season = grp_idx[0], grp_idx[1]
        if int(season) < 7:
            t_clean = re.sub(r' ', '_', t_raw.lower())
            path = "transcripts/" + decade + "/" + t_clean + "_season" + str(season) + ".txt"
            with open(path) as file:
                corpus = file.read()
            if decade == '1950s':
                episodes = re.split(pattern_50s, corpus)
            else:
                episodes = re.split(pattern_all, corpus)
            for i in range(1, len(episodes) - 1, 2):
                e = entro.getEntropy(episodes[i])
                idx = int((i - 1)/2)
                data = (t_raw, season, idx, e)
                entropy.append(data)
    return entropy

#### Processing by decade

In [70]:
entropy_1950 = obtainEntropy('1950s')

In [71]:
entropy_1950[-5:]

[('I Love Lucy', '06', 13, 4.628758542654943),
 ('I Love Lucy', '06', 14, 4.638614586908),
 ('I Love Lucy', '06', 15, 4.632009092052779),
 ('I Love Lucy', '06', 16, 4.668165611300175),
 ('I Love Lucy', '06', 17, 4.636131354295854)]

In [76]:
entropy_1960 = obtainEntropy('1960s')

In [77]:
meta['1950s'].nth(3)

Unnamed: 0 episode  \
title       season                       
I Love Lucy 01               3      03   
            02              39      04   
            03              71      04   
            04             102      04   
            05             132      04   
            06             158      04   

                                                                  url    seid  
title       season                                                             
I Love Lucy 01      view_episode_scripts.php?tv-show=i-love-lucy-1...  s01e03  
            02      view_episode_scripts.php?tv-show=i-love-lucy-1...  s02e04  
            03      view_episode_scripts.php?tv-show=i-love-lucy-1...  s03e04  
            04      view_episode_scripts.php?tv-show=i-love-lucy-1...  s04e04  
            05      view_episode_scripts.php?tv-show=i-love-lucy-1...  s05e04  
            06      view_episode_scripts.php?tv-show=i-love-lucy-1...  s06e04

In [78]:
meta['1960s'].nth(3)

Unnamed: 0 episode  \
title              season                       
Bewitched          01              28      04   
                   02              64      04   
                   03             102      04   
                   04             135      04   
                   05             168      04   
                   06             198      04   
                   07             228      04   
                   08             256      04   
Get Smart          01             384      04   
                   02             414      04   
                   03             444      04   
                   04             470      04   
                   05             496      04   
I Dream of Jeannie 01             522      04   
                   02             552      04   
                   03             583      04   
                   04             609      04   
                   05             635      04   
The Addams Family  01             282      04   
                   02             316      04   
The Flintstones    01               3      04   
The Munsters       01             346      04   

                                                                         url  \
title              season                                                      
Bewitched          01      view_episode_scripts.php?tv-show=bewitched-196...   
                   02      view_episode_scripts.php?tv-show=bewitched-196...   
                   03      view_episode_scripts.php?tv-show=bewitched-196...   
                   04      view_episode_scripts.php?tv-show=bewitched-196...   
                   05      view_episode_scripts.php?tv-show=bewitched-196...   
                   06      view_episode_scripts.php?tv-show=bewitched-196...   
                   07      view_episode_scripts.php?tv-show=bewitched-196...   
                   08      view_episode_scripts.php?tv-show=bewitched-196...   
Get Smart          01      view_episode_scripts.php?tv-show=get-smart-196...   
                   02      view_episode_scripts.php?tv-show=get-smart-196...   
                   03      view_episode_scripts.php?tv-show=get-smart-196...   
                   04      view_episode_scripts.php?tv-show=get-smart-196...   
                   05      view_episode_scripts.php?tv-show=get-smart-196...   
I Dream of Jeannie 01      view_episode_scripts.php?tv-show=i-dream-of-je...   
                   02      view_episode_scripts.php?tv-show=i-dream-of-je...   
                   03      view_episode_scripts.php?tv-show=i-dream-of-je...   
                   04      view_episode_scripts.php?tv-show=i-dream-of-je...   
                   05      view_episode_scripts.php?tv-show=i-dream-of-je...   
The Addams Family  01      view_episode_scripts.php?tv-show=the-addams-fa...   
                   02      view_episode_scripts.php?tv-show=the-addams-fa...   
The Flintstones    01      view_episode_scripts.php?tv-show=the-flintston...   
The Munsters       01      view_episode_scripts.php?tv-show=the-munsters-...   

                             seid  
title              season          
Bewitched          01      s01e04  
                   02      s02e04  
                   03      s03e04  
                   04      s04e04  
                   05      s05e04  
                   06      s06e04  
                   07      s07e04  
                   08      s08e04  
Get Smart          01      s01e04  
                   02      s02e04  
                   03      s03e04  
                   04      s04e04  
                   05      s05e04  
I Dream of Jeannie 01      s01e04  
                   02      s02e04  
                   03      s03e04  
                   04      s04e04  
                   05      s05e04  
The Addams Family  01      s01e04  
                   02      s02e04  
The Flintstones    01      s01e04  
The Munsters       01      s01e04

In [79]:
entropy_1960[-5:]

[('The Munsters', '01', 33, 4.676812141091937),
 ('The Munsters', '01', 34, 4.676069646917218),
 ('The Munsters', '01', 35, 4.660436945697844),
 ('The Munsters', '01', 36, 4.5851850160400565),
 ('The Munsters', '01', 37, 4.617652594101737)]

In [81]:
entropy_1970 = obtainEntropy('1970s')
entropy_1980 = obtainEntropy('1980s')
entropy_1990 = obtainEntropy('1990s')
entropy_2000 = obtainEntropy('2000s')
entropy_2010 = obtainEntropy('2010s')